# 🤖 Módulo 5: IA Generativa con Amazon Bedrock

**Duración**: 15 minutos  
**Objetivo**: Aprender a generar contenido científico usando IA generativa

## 🎯 ¿Qué aprenderás?
- Generar reportes científicos automáticamente
- Crear resúmenes de investigación
- Usar diferentes modelos de IA generativa
- Aplicar prompting efectivo para casos científicos
- Configurar parámetros avanzados (temperatura, top-p)

## 🔧 Configuración Inicial
Ejecuta las siguientes celdas paso a paso.

In [ ]:
# Instalar dependencias
import sys
!{sys.executable} -m pip install boto3 ipywidgets --quiet

print("✅ Dependencias instaladas")

In [ ]:
# Importar librerías
import boto3
import json
import ipywidgets as widgets
from IPython.display import display, Markdown

print("✅ Librerías importadas")

## 🔑 Configurar Credenciales AWS

In [ ]:
# Configurar Bedrock
try:
    bedrock_runtime = boto3.client('bedrock-runtime', region_name='us-east-1')
    print("✅ Cliente Bedrock configurado")
    print("⚠️ Nota: Bedrock requiere acceso especial en algunas cuentas")
except Exception as e:
    print(f"❌ Error: {e}")
    print("Configurar credenciales en la siguiente celda")
    bedrock_runtime = None

In [ ]:
# Configuración manual de credenciales
AWS_ACCESS_KEY_ID = "TU_ACCESS_KEY_AQUI"
AWS_SECRET_ACCESS_KEY = "TU_SECRET_KEY_AQUI"

if AWS_ACCESS_KEY_ID != "TU_ACCESS_KEY_AQUI":
    bedrock_runtime = boto3.client(
        'bedrock-runtime',
        aws_access_key_id=AWS_ACCESS_KEY_ID,
        aws_secret_access_key=AWS_SECRET_ACCESS_KEY,
        region_name='us-east-1'
    )
    print("✅ Credenciales configuradas manualmente")
else:
    print("⚠️ Usar credenciales por defecto")

## 📊 Prompts Científicos Predefinidos

In [ ]:
# Prompts para generación científica
SCIENTIFIC_PROMPTS = {
    "Reporte Sísmico": {
        "prompt": """Genera un reporte científico sobre actividad sísmica basado en estos datos:
        
Datos del evento:
- Magnitud: 6.2 Mw
- Profundidad: 45 km
- Ubicación: 150 km al sureste de Lima, Perú
- Fecha: 15 de noviembre de 2024
- Hora: 14:32 UTC

El reporte debe incluir:
1. Resumen ejecutivo
2. Características del evento
3. Impacto potencial
4. Recomendaciones

Usa terminología técnica apropiada pero mantenlo comprensible.""",
        "model": "anthropic.claude-3-sonnet-20240229-v1:0"
    },
    
    "Análisis Volcánico": {
        "prompt": """Crea un análisis científico sobre el monitoreo volcánico en los Andes peruanos.
        
Información base:
- Volcanes principales: Sabancaya, Ubinas, Misti
- Actividad reciente: Incremento en emisiones de gases
- Tecnologías: Cámaras térmicas, sismógrafos, análisis geoquímico
- Población en riesgo: Aproximadamente 500,000 personas

El análisis debe cubrir:
1. Estado actual de los volcanes
2. Métodos de monitoreo utilizados
3. Evaluación de riesgos
4. Estrategias de mitigación

Enfócate en aspectos técnicos y de gestión de riesgos.""",
        "model": "anthropic.claude-3-sonnet-20240229-v1:0"
    }
}

print("🤖 Prompts científicos cargados:")
for i, (title, data) in enumerate(SCIENTIFIC_PROMPTS.items(), 1):
    word_count = len(data['prompt'].split())
    print(f"  {i}. {title} ({word_count} palabras)")

## 🔍 Función de Generación con Bedrock

In [ ]:
def generate_with_bedrock(prompt, model_id, prompt_name, temperature=0.7, max_tokens=1000):
    """Genera contenido usando Amazon Bedrock con parámetros configurables"""
    try:
        print(f"🔄 Generando: {prompt_name}")
        print(f"  🤖 Modelo: {model_id.split('.')[-1]}")
        print(f"  ⚙️ Temperatura: {temperature}, Max tokens: {max_tokens}")
        
        # Preparar el cuerpo de la solicitud para Claude
        body = {
            "anthropic_version": "bedrock-2023-05-31",
            "max_tokens": max_tokens,
            "temperature": temperature,
            "messages": [
                {
                    "role": "user",
                    "content": prompt
                }
            ]
        }
        
        # Llamar a Bedrock
        response = bedrock_runtime.invoke_model(
            modelId=model_id,
            body=json.dumps(body)
        )
        
        # Procesar respuesta
        response_body = json.loads(response['body'].read())
        generated_text = response_body['content'][0]['text']
        
        print(f"✅ Contenido generado: {len(generated_text)} caracteres")
        
        return {
            'success': True,
            'generated_text': generated_text,
            'prompt_name': prompt_name,
            'model_id': model_id,
            'prompt': prompt
        }
        
    except Exception as e:
        print(f"❌ Error: {e}")
        
        # Simulación si Bedrock no está disponible
        print("🔄 Usando simulación de IA generativa...")
        
        simulated_responses = {
            "Reporte Sísmico": """# REPORTE SÍSMICO - EVENTO DEL 15 DE NOVIEMBRE 2024

## Resumen Ejecutivo
Se registró un evento sísmico de magnitud 6.2 Mw a las 14:32 UTC del 15 de noviembre de 2024, con epicentro localizado a 150 km al sureste de Lima, Perú, a una profundidad de 45 km.

## Características del Evento
- **Magnitud**: 6.2 Mw (moderado a fuerte)
- **Profundidad**: 45 km (intermedio)
- **Mecanismo**: Probable falla inversa asociada a la subducción
- **Intensidad máxima estimada**: VI-VII (Escala de Mercalli)

## Impacto Potencial
El evento pudo generar sacudidas moderadas a fuertes en un radio de 100 km. Las estructuras vulnerables en la región podrían haber experimentado daños menores.

## Recomendaciones
1. Mantener vigilancia por posibles réplicas
2. Inspeccionar infraestructura crítica
3. Activar protocolos de respuesta establecidos""",
            
            "Análisis Volcánico": """# ANÁLISIS DE MONITOREO VOLCÁNICO - ANDES PERUANOS

## Estado Actual de los Volcanes
Los volcanes Sabancaya, Ubinas y Misti mantienen diferentes niveles de actividad. El Sabancaya presenta actividad eruptiva continua con emisiones de ceniza, mientras que Ubinas y Misti se encuentran en estado de vigilancia.

## Métodos de Monitoreo
- **Sismológico**: Red de 15 estaciones sísmicas de banda ancha
- **Geoquímico**: Análisis de gases volcánicos (SO2, CO2, H2S)
- **Térmico**: Cámaras infrarrojas para detección de anomalías
- **Geodésico**: Medición de deformación del terreno

## Evaluación de Riesgos
Aproximadamente 500,000 personas viven en zonas de influencia volcánica. Los principales peligros incluyen caída de ceniza, flujos piroclásticos y lahares.

## Estrategias de Mitigación
1. Fortalecimiento del sistema de alerta temprana
2. Capacitación comunitaria en gestión de riesgos
3. Desarrollo de planes de evacuación actualizados"""
        }
        
        simulated_text = simulated_responses.get(prompt_name, f"Contenido generado para: {prompt_name}\nParámetros: temperatura={temperature}, tokens={max_tokens}")
        
        return {
            'success': True,
            'generated_text': simulated_text,
            'prompt_name': prompt_name,
            'model_id': 'simulado',
            'prompt': prompt,
            'simulated': True
        }

print("✅ Función de generación creada")

## 🚀 Ejercicio 1: Generar Reporte Sísmico

In [ ]:
# Generar reporte sísmico
prompt_name = "Reporte Sísmico"
prompt_data = SCIENTIFIC_PROMPTS[prompt_name]

print(f"📄 Generando: {prompt_name}")
print("-" * 50)

seismic_report = generate_with_bedrock(
    prompt=prompt_data['prompt'],
    model_id=prompt_data['model'],
    prompt_name=prompt_name,
    temperature=0.5,
    max_tokens=1500
)

if seismic_report.get('success'):
    print(f"\n🤖 CONTENIDO GENERADO: {seismic_report['prompt_name'].upper()}")
    print("=" * 60)
    display(Markdown(seismic_report['generated_text']))

## 🌋 Ejercicio 2: Análisis Volcánico

In [ ]:
# Generar análisis volcánico
prompt_name = "Análisis Volcánico"
prompt_data = SCIENTIFIC_PROMPTS[prompt_name]

print(f"🌋 Generando: {prompt_name}")
print("-" * 50)

volcanic_analysis = generate_with_bedrock(
    prompt=prompt_data['prompt'],
    model_id=prompt_data['model'],
    prompt_name=prompt_name,
    temperature=0.7,
    max_tokens=2000
)

if volcanic_analysis.get('success'):
    print(f"\n🤖 CONTENIDO GENERADO: {volcanic_analysis['prompt_name'].upper()}")
    print("=" * 60)
    display(Markdown(volcanic_analysis['generated_text']))

## 🎛️ Widget Interactivo

In [ ]:
# Widget para generación interactiva
def interactive_generation(prompt_selection, temperature, max_tokens):
    prompt_data = SCIENTIFIC_PROMPTS[prompt_selection]
    
    print(f"🤖 Generando: {prompt_selection}")
    print(f"⚙️ Temperatura: {temperature}, Max tokens: {max_tokens}")
    print("=" * 50)
    
    result = generate_with_bedrock(
        prompt=prompt_data['prompt'],
        model_id=prompt_data['model'],
        prompt_name=prompt_selection,
        temperature=temperature,
        max_tokens=max_tokens
    )
    
    if result.get('success'):
        print(f"\n🤖 CONTENIDO GENERADO: {result['prompt_name'].upper()}")
        print("=" * 60)
        display(Markdown(result['generated_text']))
    
    return result

# Crear widgets
prompt_selector = widgets.Dropdown(
    options=list(SCIENTIFIC_PROMPTS.keys()),
    description='Prompt:'
)

temperature_slider = widgets.FloatSlider(
    value=0.7,
    min=0.0,
    max=1.0,
    step=0.1,
    description='Temperatura:'
)

tokens_slider = widgets.IntSlider(
    value=1000,
    min=100,
    max=3000,
    step=100,
    description='Max tokens:'
)

interactive_widget = widgets.interactive(
    interactive_generation,
    prompt_selection=prompt_selector,
    temperature=temperature_slider,
    max_tokens=tokens_slider
)

display(interactive_widget)

## 🎓 Casos de Uso Reales

### Amazon Bedrock en Ciencias:
- 📄 **Generación de reportes**: Crear documentos técnicos automáticamente
- 📚 **Resúmenes de investigación**: Sintetizar literatura científica
- 💡 **Propuestas de proyectos**: Desarrollar ideas de investigación
- 🔍 **Análisis de datos**: Interpretar resultados experimentales
- 📝 **Documentación técnica**: Crear manuales y procedimientos
- 🌍 **Comunicación científica**: Adaptar contenido para diferentes audiencias

## ✅ Validación del Módulo

In [ ]:
def validate_module():
    checks = {
        "Cliente configurado": bedrock_runtime is not None,
        "Contenido generado": False,
        "Widget usado": False
    }
    
    if 'seismic_report' in globals() and seismic_report.get('success'):
        checks["Contenido generado"] = True
    
    if 'volcanic_analysis' in globals() and volcanic_analysis.get('success'):
        checks["Widget usado"] = True
    
    print("📋 VALIDACIÓN MÓDULO 5 - BEDROCK")
    print("=" * 40)
    
    for check, status in checks.items():
        icon = "✅" if status else "❌"
        print(f"{icon} {check}")
    
    completed = sum(checks.values())
    total = len(checks)
    print(f"\n📊 Progreso: {completed}/{total} ({completed/total*100:.0f}%)")
    
    if completed >= 2:
        print("\n🎉 ¡MÓDULO COMPLETADO!")
        print("➡️ Continúa con Módulo 6: Q Developer")
    else:
        print("\n⚠️ Completa los ejercicios faltantes")
    
    return completed >= 2

module_completed = validate_module()

---

## 🚀 Próximo Módulo

**💻 Módulo 6: Asistencia de Código con Amazon Q Developer**

---

*Módulo 5 de 6 completado* ✅